In [5]:
# Core Libraries:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.ticker as ticker



from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from keras.models import Model, load_model
from keras.layers import Input, Dense

In [6]:
# Loading Credit Card Fraud Dataset:

url_fraud = 'https://github.com/manugaco/ML_User_Cases/blob/master/Datasets/fraud.zip?raw=true'
zip_fraud = ZipFile(BytesIO(requests.get(url_fraud).content), 'r')
df_fraud = pd.read_csv(zip_fraud.open(zip_fraud.namelist()[0]), low_memory=False)

NameError: name 'ZipFile' is not defined